## Import 

In [25]:
import altair as alt
import pandas as pd
from classes import Nordnet
alt.data_transformers.disable_max_rows()       

DataTransformerRegistry.enable('default')

### Load Data

In [29]:
# Init nordnet class
nordnet = Nordnet()

# What share data to download. TODO: compose this list automatically from transaction data!
sharelist = [
    ["Carlsberg B A/S","DK0010181759",16099914],
    ["Ørsted","DK0060094928",16557154],
    ["Demant A/S","DK0060738599", 16099933],
    ]

# Download transactions and prices
nordnet.get_transactions(startdate='2020-06-11')
nordnet.get_share_prices(sharelist, startdate='2020-06-11')

# Load data into Pandas Dataframe
df_trans = pd.read_csv('./data/transactions.csv',sep=';', decimal=",")
df_prices = pd.read_csv('./data/prices.csv', sep=';', decimal=",")
 


### Clean Data

In [30]:
# Convert date-row to datetime (for nice plotting)
df_prices['date'] = pd.to_datetime(df_prices['date'], format='%Y-%m-%d')

# Choose columns to work with
df_trans = df_trans[['Handelsdag', 'Transaktionstype', 'Værdipapirer', 'Værdipapirtype', 'Kurs', 'Antal', 'Indkøbsværdi', 'Valuta', 'cost', 'Saldo', 'Vekslingskurs', 'Resultat', 'ISIN']]

# Rename columns
df_trans = df_trans.rename(columns={'Handelsdag': 'date', 
                                    'Transaktionstype': 'transaction_type', 
                                    'Værdipapirer': 'share', 
                                    'Værdipapirtype': 'type', 
                                    'Kurs': 'price', 
                                    'Valuta': 'currency',
                                    'Antal': 'quantity', 
                                    'Indkøbsværdi': 'purchase_value', 
                                    'Saldo': 'balance',
                                    'Vekslingskurs': 'exchage_rate',
                                    'Resultat': 'result', 
                                    'ISIN': 'isin'})

# Convert Saldo column to float
def handle_comma(x):
    x = x.replace('.', '')
    x = x.replace(',', '.')
    return x

df_trans['balance'] = df_trans['balance'].apply(lambda x: handle_comma(x))
df_trans['balance'] = df_trans['balance'].astype(float)


In [ ]:
## TODO:
# 1. Create cost column ( to keep track of kurtage spending)
# 2. Compose sharelist from transactions dataframe to avoid to manually typing it in.
# 3. Plot: - market value
#          - money invested
#          - expenses
#          - profit


# Add cost column to hold brokerage information
df_trans['cost'] = 0


### Plot prices 

In [38]:
import altair as alt
alt.Chart(df_prices).mark_line().encode(
    x='date',
    y='price',
    color='share'
).properties(
    width=800,
    height=400
).interactive()

alt.Chart(...)